In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk

from sklearn.preprocessing import LabelEncoder

import tensorflow_hub as hub
import tensorflow as tf
from tensorflow import keras
import tensorflow.compat.v2.feature_column as fc
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.4 MB/s eta 0:00:00


In [ ]:
from transformers import TFBertModel
import transformers

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Deep Learning/NLP Vietnamese/synthetic_train.csv')
df_val = pd.read_csv('/content/drive/MyDrive/Deep Learning/NLP Vietnamese/synthetic_val.csv')

In [ ]:
df_train.head()

,sentence,sentiment,topic
0,Đội ngũ bảo trì quá thưa thớt dẫn đến không đả...,negative,facility
1,The university's musical and artistic faciliti...,neutral,facility
2,Phương pháp giảng dạy phù hợp với các đối tượn...,neutral,curriculum
3,Chương trình học giúp tôi trở thành một chuyên...,positive,curriculum
4,Tôi nghĩ rằng chương trình đào tạo có thể có t...,neutral,curriculum


In [ ]:
df_train.shape

(8144, 3)

In [ ]:
df_train.isnull().sum()

sentence     0
sentiment    0
topic        0
dtype: int64

In [ ]:
df_train.nunique()

sentence     8144
sentiment       3
topic           4
dtype: int64

# Loại bỏ ký tự đặc biệt trong câu


In [ ]:
def preprocess_text(text):
    # Loại bỏ các ký tự không mong muốn
    text = text.replace(",", "")
    text = text.replace(".", "")
    text = text.replace("'", "")
    text = text.replace("\"", "")
    return text

df_train["sentence"] = df_train["sentence"].apply(preprocess_text)
df_val["sentence"] = df_val["sentence"].apply(preprocess_text)

# Gắn nhãn cho từng từ

In [ ]:
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(df_train['sentence'])
word_index = tokenizer.word_index
print(word_index)

{'<OOV>': 1, 'và': 2, 'viên': 3, 'học': 4, 'sinh': 5, 'của': 6, 'rất': 7, 'không': 8, 'có': 9, 'giảng': 10, 'tôi': 11, 'các': 12, 'trường': 13, 'được': 14, 'tập': 15, 'trong': 16, 'người': 17, 'cho': 18, 'giúp': 19, 'này': 20, 'trình': 21, 'với': 22, 'việc': 23, 'một': 24, 'năng': 25, 'thầy': 26, 'để': 27, 'sự': 28, 'luôn': 29, 'là': 30, 'tạo': 31, 'ấy': 32, 'dạy': 33, 'cô': 34, 'những': 35, 'bạn': 36, 'tốt': 37, 'giáo': 38, 'anh': 39, 'chương': 40, 'cảm': 41, 'nhiều': 42, 'thực': 43, 'phòng': 44, 'mình': 45, 'quá': 46, 'thể': 47, 'thấy': 48, 'đủ': 49, 'kiến': 50, 'cách': 51, 'khác': 52, 'bị': 53, 'ra': 54, 'động': 55, 'công': 56, 'tâm': 57, 'thức': 58, 'kỹ': 59, 'hiểu': 60, 'hơn': 61, 'thiết': 62, 'thường': 63, 'đầy': 64, 'khó': 65, 'the': 66, 'khi': 67, 'phát': 68, 'giải': 69, 'làm': 70, 'triển': 71, 'nghiệp': 72, 'đến': 73, 'môn': 74, 'hợp': 75, 'hoạt': 76, 'về': 77, 'quan': 78, 'thích': 79, 'bài': 80, 'chất': 81, 'tại': 82, 'tế': 83, 'đào': 84, 'dễ': 85, 'vực': 86, 'thiếu': 87, 'tì

# Thực hiện gắn nhãn và chuyển về vector


In [ ]:
training_sequences = tokenizer.texts_to_sequences(df_train['sentence'])
training_padded = pad_sequences(training_sequences, maxlen=20, padding='post', truncating='post')

validating_sequences = tokenizer.texts_to_sequences(df_val['sentence'])
validating_padded = pad_sequences(validating_sequences, maxlen=20, padding='post', truncating='post')

In [ ]:
training_padded

array([[ 237,  436,  127, ...,   62,   53,    0],
       [  66,  499, 1674, ...,    0,    0,    0],
       [ 200,  182,   10, ...,    0,    0,    0],
       ...,
       [  39,  141,  164, ...,   11,    0,    0],
       [  40,   21,    4, ...,    0,    0,    0],
       [  11,  104,  629, ...,   11,    0,    0]], dtype=int32)

In [ ]:
validating_padded

array([[ 81, 125, 170, ...,   0,   0,   0],
       [267, 372,   4, ...,   0,   0,   0],
       [ 13,  11,  87, ...,  22, 160, 346],
       ...,
       [ 87, 117, 358, ...,   0,   0,   0],
       [ 34,  32,   7, ...,   0,   0,   0],
       [ 39,  32,   9, ...,  93,  19, 107]], dtype=int32)

In [ ]:
pd.concat((df_train['sentiment'],df_val['sentiment']), axis=0)

0       negative
1        neutral
2        neutral
3       positive
4        neutral
          ...   
2031     neutral
2032     neutral
2033    negative
2034    positive
2035    positive
Name: sentiment, Length: 10180, dtype: object

# Gắn nhãn cho cột "setiment"

In [ ]:
le = LabelEncoder()
le.fit(pd.concat((df_train['sentiment'],df_val['sentiment']), axis=0))

LabelEncoder()

In [ ]:
df_train['sentiment'] = le.transform(df_train['sentiment'])
df_val['sentiment'] = le.transform(df_val['sentiment'])

In [ ]:
df = pd.concat((df_train,df_val), axis=0).sample(42)

In [ ]:
df['sentiment'] = le.transform(df['sentiment'])

In [ ]:
training_padded = np.array(training_padded)
validating_padded = np.array(validating_padded)

In [ ]:
# text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)

# preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
# encoder_inputs = preprocessor(df_train['sentence'])

# encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/2",trainable=True)
# outputs = encoder(df_train['sentiment'])

# pooled_output = outputs["pooled_output"]    # [batch_size, 128].
# sequence_output = outputs["sequence_output"]    # [batch_size, seq_length, 128].

# Train trên LSTM model

- Tạo 1 lớp Embedding để chuyển ma trận chữ về các vector.
- Tạo 1 lớp LSTM model để train.
- Tạo 4 lớp ẩn (Dense) có chứa những hàm kích hoạt, ở đây là hàm "relu".
- Tạo lớp cuối cùng có sử dụng hàm "sigmoid" để phân biệt câu và dự đoán câu.
- Có thể tăng hoặc giảm số lượng lớp Dense. Tùy thuộc vào mô hình. Càng nhiều lớp thì mô hình càng nặng. Tương tự như lớp, thì số node trong lớp có thể tăng hoặc giảm.

In [ ]:
vocab_size = 10000
embedding_dim = 16
max_length = 20
training_size = 20000


model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    #tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.LSTM(64),
    #TFBertModel.from_pretrained("albert-base-v2"),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='sigmoid')
])
model.summary()

model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          160000    
                                                                 
 lstm (LSTM)                 (None, 64)                20736     
                                                                 
 dense_4 (Dense)             (None, 128)               8320      
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dense_7 (Dense)             (None, 16)                528       
                                                                 
 dense_8 (Dense)             (None, 3)                

In [ ]:
history = model.fit(training_padded, df_train['sentiment'].values, epochs = 100, validation_data=(validating_padded, df_val['sentiment'].values), verbose=2)

Epoch 1/100
255/255 - 20s - loss: 0.7414 - accuracy: 0.6181 - val_loss: 0.5326 - val_accuracy: 0.7593 - 20s/epoch - 78ms/step
Epoch 2/100
255/255 - 3s - loss: 0.4730 - accuracy: 0.7818 - val_loss: 0.4932 - val_accuracy: 0.7868 - 3s/epoch - 13ms/step
Epoch 3/100
255/255 - 2s - loss: 0.4015 - accuracy: 0.8194 - val_loss: 0.4652 - val_accuracy: 0.7981 - 2s/epoch - 9ms/step
Epoch 4/100
255/255 - 2s - loss: 0.3717 - accuracy: 0.8358 - val_loss: 0.4686 - val_accuracy: 0.8070 - 2s/epoch - 8ms/step
Epoch 5/100
255/255 - 3s - loss: 0.3531 - accuracy: 0.8439 - val_loss: 0.5038 - val_accuracy: 0.7755 - 3s/epoch - 12ms/step
Epoch 6/100
255/255 - 2s - loss: 0.3330 - accuracy: 0.8540 - val_loss: 0.4816 - val_accuracy: 0.8035 - 2s/epoch - 7ms/step
Epoch 7/100
255/255 - 2s - loss: 0.3236 - accuracy: 0.8567 - val_loss: 0.5204 - val_accuracy: 0.8006 - 2s/epoch - 6ms/step
Epoch 8/100
255/255 - 2s - loss: 0.3114 - accuracy: 0.8630 - val_loss: 0.5615 - val_accuracy: 0.8065 - 2s/epoch - 8ms/step
Epoch 9/100

In [ ]:
##########################################################################################################################################

In [ ]:
pip install tensorflow_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 48.8 MB/s eta 0:00:00


In [ ]:
import tensorflow_text as text

In [ ]:
bert_base = transformers.AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Train trên bert_base_uncased

- Sử dụng mô hình có sẵn trên tensorflow
- Mô hình này sẽ sử lý sẵn nên không cần sẵn lý dữ liệu trước đó.
- Tạo 1 lớp dropout, lớp ẩn(Dense) chứa hàm kích hoạt, lớp cuối cùng dùng hàm softmax để dự đoán và học.
- Có thể tăng hoặc giảm số lượng lớp Dense. Tùy thuộc vào mô hình. Càng nhiều lớp thì mô hình càng nặng. Tương tự như lớp, thì số node trong lớp có thể tăng hoặc giảm.


In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(32, activation='relu')(l)
l = tf.keras.layers.Dense(16, activation='relu')(l)
l = tf.keras.layers.Dense(3, activation='softmax', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer_9 (KerasLayer)  {'input_mask': (None, 128)   0         ['text[0][0]']                
                             , 'input_word_ids': (None,                                           
                              128),                                                               
                              'input_type_ids': (None,                                            
                             128)}                                                                
                                                                                              

In [ ]:
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
# METRICS = [
#       tf.keras.metrics.BinaryAccuracy(name='accuracy'),
#       tf.keras.metrics.Precision(name='precision'),
#       tf.keras.metrics.Recall(name='recall')
# ]

# model.compile(optimizer='adam',
#               loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
#               metrics=METRICS)

In [ ]:
X = df['sentence'].values
y = df['sentiment'].values

In [ ]:
history = model.fit(X, y, epochs = 100, validation_split=0.3, verbose=2)

Epoch 1/100
1/1 - 33s - loss: 1.3382 - accuracy: 0.1724 - val_loss: 1.3208 - val_accuracy: 0.1538 - 33s/epoch - 33s/step
Epoch 2/100
1/1 - 25s - loss: 1.1255 - accuracy: 0.4483 - val_loss: 1.3843 - val_accuracy: 0.1538 - 25s/epoch - 25s/step
Epoch 3/100
1/1 - 20s - loss: 1.0761 - accuracy: 0.4483 - val_loss: 1.3655 - val_accuracy: 0.1538 - 20s/epoch - 20s/step
Epoch 4/100
1/1 - 21s - loss: 1.0632 - accuracy: 0.4828 - val_loss: 1.2462 - val_accuracy: 0.1538 - 21s/epoch - 21s/step
Epoch 5/100
1/1 - 24s - loss: 1.0186 - accuracy: 0.4828 - val_loss: 1.1450 - val_accuracy: 0.2308 - 24s/epoch - 24s/step
Epoch 6/100
1/1 - 24s - loss: 1.0299 - accuracy: 0.4828 - val_loss: 1.1001 - val_accuracy: 0.5385 - 24s/epoch - 24s/step
Epoch 7/100
1/1 - 25s - loss: 0.9807 - accuracy: 0.5172 - val_loss: 1.0836 - val_accuracy: 0.5385 - 25s/epoch - 25s/step
Epoch 8/100
1/1 - 19s - loss: 1.0535 - accuracy: 0.4483 - val_loss: 1.1358 - val_accuracy: 0.4615 - 19s/epoch - 19s/step
Epoch 9/100
1/1 - 25s - loss: 0.

In [ ]:
#######################################################################################################################################

# Phần này bỏ

In [ ]:
# albert_module = hub.load("https://tfhub.dev/google/albert_base/3")
# #albert_module.signatures['default'](['<OOV>', 'và'])
# albert_inputs = dict(input_ids=word_index)

# albert_outputs = albert_module(albert_inputs, signature="<OOV>", as_dict=True)

# pooled_output = albert_outputs["pooled_output"]
# sequence_output = albert_outputs["sequence_output"]

In [ ]:
hub_layer = hub.KerasLayer("https://tfhub.dev/google/albert_base/3",input_shape=[], dtype=tf.string)

model = keras.Sequential()
model.add(hub_layer)
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(3, activation='sigmoid'))

model.summary()


In [ ]:
##################################################################################################################################################

# Train trên NNLM model
- Mô hình không có xử lý sẵn dữ liệu nên sẽ phải xử lý dữ liệu trước đó, sau đó mới đưa vào mô hình để train.
- Tạo 3 lớp Dense và 1 lớp có hàm kích hoạt "sigmoid" để dự máy có thể học và dự đoán.
- Có thể tăng hoặc giảm số lượng lớp Dense. Tùy thuộc vào mô hình. Càng nhiều lớp thì mô hình càng nặng. Tương tự như lớp, thì số node trong lớp có thể tăng hoặc giảm.

In [ ]:
hub_layer = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim50/2",input_shape=[], dtype=tf.string)

model = keras.Sequential()
model.add(hub_layer)
#model.add(keras.layers.Dense(64, activation='relu'))
#model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(3, activation='sigmoid'))

model.summary()


In [ ]:
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])

In [ ]:
X = df['sentence'].values
y = df['sentiment'].values

In [ ]:
history = model.fit(X, y, epochs = 100, validation_split=0.3, verbose=2)

Epoch 1/100
1/1 - 4s - loss: 1.0993 - accuracy: 0.4138 - val_loss: 1.1152 - val_accuracy: 0.1538 - 4s/epoch - 4s/step
Epoch 2/100
1/1 - 0s - loss: 1.0903 - accuracy: 0.4138 - val_loss: 1.1154 - val_accuracy: 0.1538 - 117ms/epoch - 117ms/step
Epoch 3/100
1/1 - 0s - loss: 1.0825 - accuracy: 0.4483 - val_loss: 1.1145 - val_accuracy: 0.1538 - 85ms/epoch - 85ms/step
Epoch 4/100
1/1 - 0s - loss: 1.0756 - accuracy: 0.4483 - val_loss: 1.1141 - val_accuracy: 0.1538 - 81ms/epoch - 81ms/step
Epoch 5/100
1/1 - 0s - loss: 1.0682 - accuracy: 0.4828 - val_loss: 1.1138 - val_accuracy: 0.1538 - 97ms/epoch - 97ms/step
Epoch 6/100
1/1 - 0s - loss: 1.0602 - accuracy: 0.4828 - val_loss: 1.1137 - val_accuracy: 0.1538 - 95ms/epoch - 95ms/step
Epoch 7/100
1/1 - 0s - loss: 1.0512 - accuracy: 0.4828 - val_loss: 1.1149 - val_accuracy: 0.1538 - 76ms/epoch - 76ms/step
Epoch 8/100
1/1 - 0s - loss: 1.0416 - accuracy: 0.4828 - val_loss: 1.1164 - val_accuracy: 0.1538 - 124ms/epoch - 124ms/step
Epoch 9/100
1/1 - 0s - l

In [ ]:
##############################################################################################################################################################

# Phần này bỏ

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline, DataCollatorForLanguageModeling, TrainingArguments, Trainer

pipe = pipeline("fill-mask", model="albert-base-v2")

# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")
model = AutoModelForMaskedLM.from_pretrained("albert-base-v2")

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForMaskedLM: ['albert.pooler.bias', 'albert.pooler.weight']
- This IS expected if you are initializing AlbertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForMaskedLM: ['albert.pooler.bias', 'albert.pooler.weight']
- This IS expected if you are initializing AlbertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.resize_token_embeddings(len(tokenizer))
tokenizer.pad_token = tokenizer.eos_token

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 30000. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


In [ ]:
# Data collator for padding
data_collator = DataCollatorForLanguageModeling(tokenizer,mlm=False,return_tensors="pt")

In [ ]:
# Load dataset
train_dataset = df['sentence'].values
test_dataset = df['sentiment'].values

In [ ]:
import math

batch_size=6
gradient_accumulation_steps=2

step_per_batch = math.ceil(len(train_dataset) / (batch_size * gradient_accumulation_steps))
print('step_per_batch',step_per_batch)

step_per_batch 4


In [ ]:
#pip install torch torchvision torchaudio

In [ ]:
def train(model):
    training_args = TrainingArguments(
        output_dir='training_article',
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        evaluation_strategy='steps',
        num_train_epochs=30,
        save_steps=step_per_batch,
        eval_steps=step_per_batch,
        logging_steps=step_per_batch,
        learning_rate=0.001,
        warmup_steps=step_per_batch * 5,
        save_total_limit=5,
        load_best_model_at_end=True,
        prediction_loss_only=True,
        metric_for_best_model='loss',
    )
    trainer = Trainer(
        model=model,
        data_collator=data_collator,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
    )

    trainer.place_model_on_device=False
    trainer.train()
    trainer.save_model('training_article/best_model')
    tokenizer.save_pretrained("training_article/best_model")

train(model)

ImportError: ignored